## Part3: The Problem of Fairness 

In [ ]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from itertools import combinations
from pathlib import Path


POP_FILE   = Path("Project 1 Data- version 0.0- Yue Chen/population_calculated.csv")
CLASS_FILE = Path("Project 1 Data- version 0.0- Yue Chen/demand_classification.csv")
FAC_FILE   = Path("Project 1 Data- version 0.0- Yue Chen/child_care_regulated_cleaned.csv")
SITE_FILE  = Path("Project 1 Data- version 0.0- Yue Chen/potential_locations.csv")


BUDGET_B = 100_000_000        
W0_5 = 2.0 / 3.0
W5_12 = 1.0 / 3.0
BETA = 100.0                  
DELTA = 0.06                  
Q05_IMPUTE_RATIO = 1

DEFAULT_SIZES = pd.DataFrame({
    "k": ["S", "M", "L"],
    "cap_k": [100, 200, 400],
    "cap_k_0_5": [50, 100, 200],
    "cost_k": [65000, 95000, 115000],
})

# ------------------------------
# Helper functions (copied/compatible with your Part2)
# ------------------------------
def pad_zip(z):
    if pd.isna(z):
        return ""
    s = str(z).strip()
    if s.endswith(".0"):
        s = s[:-2]
    return s.zfill(5)

def haversine(lat1, lon1, lat2, lon2):
    if any(pd.isna(v) for v in (lat1, lon1, lat2, lon2)):
        return np.inf
    R = 3958.8
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    a = np.sin((lat2 - lat1) / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin((lon2 - lon1) / 2.0)**2
    return 2.0 * R * np.arcsin(np.sqrt(a))

def cost_segments(n):
    if n <= 0 or pd.isna(n):
        large = 1e6
        return (large, large, large)
    return ((20000 + 200.0 * n) / n,
            (20000 + 400.0 * n) / n,
            (20000 + 1000.0 * n) / n)

# ------------------------------
# Data loading (kept compatible with your part2 load_data)
# ------------------------------
def load_data():
    pop = pd.read_csv(POP_FILE)
    pop["zip_code_cleaned"] = pop["zip_code_cleaned"].apply(pad_zip)
    pop.rename(columns={"Population_0_12": "pop_0_12", "Population_0_5": "pop_0_5"}, inplace=True)
    pop["pop_0_12"] = pd.to_numeric(pop.get("pop_0_12", 0), errors="coerce").fillna(0)
    pop["pop_0_5"]  = pd.to_numeric(pop.get("pop_0_5", 0), errors="coerce").fillna(0)

    cls = pd.read_csv(CLASS_FILE)
    cls["zip_code_cleaned"] = cls["zip_code_cleaned"].apply(pad_zip)
    cls.rename(columns={"Demand_Classification": "class"}, inplace=True)
    cls["class"] = cls["class"].astype(str).str.strip().str.lower()
    cls["t_i"] = np.where(cls["class"].str.contains("high", na=False), 0.5, 1/3)

    common_zips = set(pop["zip_code_cleaned"]) & set(cls["zip_code_cleaned"])
    print(f"[load_data] ZIP intersection: {len(common_zips)} common of (pop={len(pop)}, cls={len(cls)})")

    pop = pop[pop["zip_code_cleaned"].isin(common_zips)].copy()
    cls = cls[cls["zip_code_cleaned"].isin(common_zips)].copy()
    
    areas = pop.merge(cls[["zip_code_cleaned", "t_i"]], on="zip_code_cleaned", how="inner")
    areas["pop_0_12"] = areas["pop_0_12"].fillna(0)
    areas["pop_0_5"] = areas["pop_0_5"].fillna(0)
    areas["t_i"] = areas["t_i"].fillna(1/3)

    fac = pd.read_csv(FAC_FILE)
    fac["zip_code_cleaned"] = fac["zip_code_cleaned"].apply(pad_zip)
    if "total_capacity" in fac.columns:
        fac.rename(columns={"total_capacity": "n_f"}, inplace=True)
    if "latitude" in fac.columns:
        fac.rename(columns={"latitude": "lat"}, inplace=True)
    if "longitude" in fac.columns:
        fac.rename(columns={"longitude": "lon"}, inplace=True)

    if "q_0_5" not in fac.columns:
        if all(col in fac.columns for col in ["infant_capacity", "toddler_capacity", "preschool_capacity"]):
            fac["q_0_5"] = fac["infant_capacity"].fillna(0) + fac["toddler_capacity"].fillna(0) + fac["preschool_capacity"].fillna(0)
        else:
            fac["n_f"] = pd.to_numeric(fac.get("n_f", 0), errors="coerce").fillna(0)
            fac["q_0_5"] = (fac["n_f"] * Q05_IMPUTE_RATIO).round(0)

    fac = fac[["facility_id", "zip_code_cleaned", "n_f", "q_0_5", "lat", "lon"]].copy()
    fac["n_f"] = pd.to_numeric(fac["n_f"], errors="coerce").fillna(0)
    fac["q_0_5"] = pd.to_numeric(fac["q_0_5"], errors="coerce").fillna(0)
    before_fac_count = len(fac)
    fac = fac[fac["n_f"] > 0].copy()
    after_fac_count = len(fac)

    site = pd.read_csv(SITE_FILE)
    site.rename(columns={"zipcode": "zip_code_cleaned", "latitude": "lat", "longitude": "lon"}, inplace=True)
    site["zip_code_cleaned"] = site["zip_code_cleaned"].apply(pad_zip)
    site["site_id"] = np.arange(1, len(site) + 1)
    site = site[["site_id", "zip_code_cleaned", "lat", "lon"]].copy()

    sizes = DEFAULT_SIZES.copy()

    drop_zips = ["11219"]
    areas = areas[~areas["zip_code_cleaned"].isin(drop_zips)].copy()
    print(f"[load_data] Dropped infeasible ZIPs: {drop_zips}")
    print(f"[load_data] areas: {len(areas)}, facilities (before filter): {before_fac_count}, (after filter): {after_fac_count}, sites: {len(site)}")
    return areas, fac, site, sizes

# ------------------------------
# Spatial helpers (compatible)
# ------------------------------
def infer_sites(sites, facs, delta=DELTA):
    sites = sites.copy()
    sites["allowed"] = 1
    if len(facs) == 0:
        sites["allowed"] = 1
        return sites
    for i, s in sites.iterrows():
        zf = facs[facs["zip_code_cleaned"] == s["zip_code_cleaned"]]
        infeasible = False
        for _, f in zf.iterrows():
            if haversine(s["lat"], s["lon"], f["lat"], f["lon"]) < delta:
                infeasible = True
                break
        sites.loc[i, "allowed"] = 0 if infeasible else 1
    return sites

def conflict_pairs(sites, delta=DELTA):
    pairs = []
    if len(sites) == 0:
        return pairs
    for zip_i, grp in sites.groupby("zip_code_cleaned"):
        idxs = list(grp.index)
        if len(idxs) < 2:
            continue
        for i, j in combinations(idxs, 2):
            if haversine(grp.loc[i, "lat"], grp.loc[i, "lon"], grp.loc[j, "lat"], grp.loc[j, "lon"]) < delta:
                pairs.append((grp.loc[i, "site_id"], grp.loc[j, "site_id"]))
    print(f"[conflict_pairs] conflict pairs found: {len(pairs)}")
    return pairs

# ------------------------------
# Build fairness model 
# ------------------------------
def build_fairness_model(areas, facs, sites, sizes, pairs, time_limit=300, mip_gap=0.01, B=BUDGET_B):
    
    I = list(areas["zip_code_cleaned"])
    F = list(facs["facility_id"])
    S = list(sites["site_id"])
    K = list(sizes["k"])


    nf   = dict(zip(facs.facility_id, facs.n_f))
    q05  = dict(zip(facs.facility_id, facs.q_0_5))
    zip_f = dict(zip(facs.facility_id, facs.zip_code_cleaned))
    zip_s = dict(zip(sites.site_id, sites.zip_code_cleaned))
    ti   = dict(zip(areas.zip_code_cleaned, areas.t_i))
    Pi   = dict(zip(areas.zip_code_cleaned, areas.pop_0_12))
    Pi05 = dict(zip(areas.zip_code_cleaned, areas.pop_0_5))
    cap  = dict(zip(sizes.k, sizes.cap_k))
    cap05= dict(zip(sizes.k, sizes.cap_k_0_5))
    cost = dict(zip(sizes.k, sizes.cost_k))
    allow= dict(zip(sites.site_id, sites.allowed if "allowed" in sites.columns else [1]*len(sites)))

    cap = {k: float(v) for k, v in cap.items()}
    cap05 = {k: float(v) for k, v in cap05.items()}
    cost = {k: float(v) for k, v in cost.items()}

    c1 = {f: cost_segments(nf.get(f, 0))[0] for f in F}
    c2 = {f: cost_segments(nf.get(f, 0))[1] for f in F}
    c3 = {f: cost_segments(nf.get(f, 0))[2] for f in F}

    m = gp.Model("Part3_Fairness")
    m.Params.OutputFlag = 1
    m.setParam("TimeLimit", time_limit)
    m.setParam("MIPGap", mip_gap)

    x1 = m.addVars(F, lb=0.0, name="x1") if F else {}
    x2 = m.addVars(F, lb=0.0, name="x2") if F else {}
    x3 = m.addVars(F, lb=0.0, name="x3") if F else {}
    y1 = m.addVars(F, lb=0.0, ub=1.0, name="y1") if F else {}
    y2 = m.addVars(F, lb=0.0, ub=1.0, name="y2") if F else {}
    y3 = m.addVars(F, lb=0.0, ub=1.0, name="y3") if F else {}
    xF = m.addVars(F, lb=0.0, name="xF") if F else {}
    z05 = m.addVars(F, lb=0.0, name="z05") if F else {}

    y_sk = m.addVars(S, K, vtype=GRB.BINARY, name="y_sk") if S and K else {}
    u_sk05 = m.addVars(S, K, lb=0.0, name="u_sk05") if S and K else {}

    r05 = m.addVars(I, lb=0.0, ub=1.0, name="r05") if I else {}
    r512 = m.addVars(I, lb=0.0, ub=1.0, name="r512") if I else {}
    g = m.addVars(I, lb=0.0, ub=1.0, name="g") if I else {}

    g_min = m.addVar(lb=0.0, ub=1.0, name="g_min")
    g_max = m.addVar(lb=0.0, ub=1.0, name="g_max")

    Ctot = {}
    C05 = {}
    C512 = {}

    for i in I:
        expr_tot = gp.LinExpr()
        expr_05 = gp.LinExpr()

        for f in F:
            if zip_f.get(f, "") == i:
                expr_tot += nf.get(f, 0.0)
                expr_tot += xF[f]
                expr_05 += q05.get(f, 0.0)
                expr_05 += z05[f]

        for s in S:
            if zip_s.get(s, "") == i:
                for k in K:
                    expr_tot += cap[k] * y_sk[s, k]
                    expr_05 += u_sk05[s, k]

        Ctot[i] = expr_tot
        C05[i] = expr_05
        C512[i] = expr_tot - expr_05


    # (a) piecewise expansion definitions and ordering (as in Section 5)
    for f in F:
        n = float(nf.get(f, 0.0))
        m.addConstr(x1[f] == 0.10 * n * y1[f], name=f"x1_def_{f}")
        m.addConstr(x2[f] == 0.05 * n * y2[f], name=f"x2_def_{f}")
        m.addConstr(x3[f] == 0.05 * n * y3[f], name=f"x3_def_{f}")
        m.addConstr(y2[f] <= y1[f], name=f"y2_le_y1_{f}")
        m.addConstr(y3[f] <= y2[f], name=f"y3_le_y2_{f}")
        m.addConstr(xF[f] == x1[f] + x2[f] + x3[f], name=f"xF_def_{f}")
        m.addConstr(xF[f] <= 0.20 * n, name=f"xF_ub_{f}")
        m.addConstr(z05[f] <= xF[f], name=f"z05_le_xF_{f}")

    # (b) site feasibility & 0-5 allocation
    for s in S:
        if allow.get(s, 1) == 0:
            m.addConstr(gp.quicksum(y_sk[s, k] for k in K) == 0, name=f"site_block_{s}")
        else:
            m.addConstr(gp.quicksum(y_sk[s, k] for k in K) <= 1, name=f"site_one_size_{s}")
        for k in K:
            m.addConstr(u_sk05[s, k] <= cap05[k] * y_sk[s, k], name=f"u_cap05_{s}_{k}")

    # (c) coverage-ratio identities (linear) and bounds
    for i in I:
    
        Pi05_i = float(Pi05.get(i, 0.0))
        Pi512_i = float(max(Pi.get(i, 0.0) - Pi05_i, 0.0))
        Pi_tot_i = float(Pi.get(i, 0.0))

        # P^{0–5} * r05 = C05
        if Pi05_i > 0:
            m.addConstr(r05[i] * Pi05_i == C05[i], name=f"covid_identity_05_{i}")
        else:
            m.addConstr(C05[i] == 0.0, name=f"zero_pop_05_cap_{i}")
            m.addConstr(r05[i] == 0.0, name=f"zero_pop_05_ratio_{i}")

        # P^{5–12} * r512 = C512
        if Pi512_i > 0:
            m.addConstr(r512[i] * Pi512_i == C512[i], name=f"covid_identity_512_{i}")
        else:
            m.addConstr(C512[i] == 0.0, name=f"zero_pop_512_cap_{i}")
            m.addConstr(r512[i] == 0.0, name=f"zero_pop_512_ratio_{i}")

        # Total: P * g_i = Ctot
        if Pi_tot_i > 0:
            m.addConstr(g[i] * Pi_tot_i == Ctot[i], name=f"cov_identity_tot_{i}")
        else:
            m.addConstr(Ctot[i] == 0.0, name=f"zero_pop_tot_cap_{i}")
            m.addConstr(g[i] == 0.0, name=f"zero_pop_tot_ratio_{i}")


    # (d) no-desert policy targets (Eq. 18)
    for i in I:
        Pi_tot_i = float(Pi.get(i, 0.0))
        Pi05_i = float(Pi05.get(i, 0.0))
        ti_val = float(ti.get(i, 1/3))

        m.addConstr(Ctot[i] >= ti_val * Pi_tot_i, name=f"no_desert_tot_{i}")

        m.addConstr(C05[i] >= (2.0 / 3.0) * Pi05_i, name=f"no_desert_05_{i}")

    # (e) fairness gap constraints (Eq. 19–20)
    for i in I:
        m.addConstr(g_min <= g[i], name=f"gmin_le_g_{i}")
        m.addConstr(g[i] <= g_max, name=f"g_le_gmax_{i}")

    m.addConstr(g_max - g_min <= 0.1, name="max_gap_limit")

    # (f) global budget (Eq. 21)
    budget_expr = gp.LinExpr()

    for f in F:
        budget_expr += c1.get(f, 0.0) * x1[f]
        budget_expr += c2.get(f, 0.0) * x2[f]
        budget_expr += c3.get(f, 0.0) * x3[f]

    for s in S:
        for k in K:
            budget_expr += cost[k] * y_sk[s, k]

    for f in F:
        budget_expr += BETA * z05[f]
    for s in S:
        for k in K:
            budget_expr += BETA * u_sk05[s, k]

    m.addConstr(budget_expr <= B, name="global_budget")

    # (g) site-site conflict pairs (as in Section 5)
    for (s1, s2) in pairs:
        for k1 in K:
            for k2 in K:
                m.addConstr(y_sk[s1, k1] + y_sk[s2, k2] <= 1, name=f"conf_{s1}_{s2}_{k1}_{k2}")

    # ---- Objective: maximize social coverage index (Eq. 15) ----
    objective = gp.quicksum(W0_5 * r05[i] + W5_12 * r512[i] for i in I)
    m.setObjective(objective, GRB.MAXIMIZE)

    new_vars = {
        "r05": r05, "r512": r512, "g": g,
        "g_min": g_min, "g_max": g_max,
        "Ctot": Ctot, "C05": C05, "C512": C512
    }
    return m, (xF, z05, y_sk, u_sk05, x1, x2, x3), new_vars

# ------------------------------
# Main
# ------------------------------
def main():
    areas, facs, sites, sizes = load_data()
    sites = infer_sites(sites, facs)
    pairs = conflict_pairs(sites)

    print("[main] Building fairness model (Section 6)...")
    m, core_vars, new_vars = build_fairness_model(areas, facs, sites, sizes, pairs,
                                                  time_limit=300, mip_gap=0.005, B=BUDGET_B)
    print("[main] Optimizing fairness model...")
    m.optimize()

    status = m.status
    if status in (GRB.Status.OPTIMAL, GRB.Status.TIME_LIMIT, GRB.Status.SUBOPTIMAL):
        print(f"[main] Solver status {status}, Obj (social coverage) = {m.objVal:.6f}")

        print(f"g_min = {new_vars['g_min'].X:.4f}, g_max = {new_vars['g_max'].X:.4f}")
        I = list(areas["zip_code_cleaned"])
        for i in I:
            r05v = new_vars["r05"][i].X if i in new_vars["r05"] else float("nan")
            r512v = new_vars["r512"][i].X if i in new_vars["r512"] else float("nan")
            gv = new_vars["g"][i].X if i in new_vars["g"] else float("nan")
            Ctot_i = new_vars["Ctot"][i].getValue()
            C05_i = new_vars["C05"][i].getValue()
            print(f"Area {i}: Ctot={Ctot_i:.1f}, C0-5={C05_i:.1f}, r0-5={r05v:.4f}, r5-12={r512v:.4f}, g={gv:.4f}")

        xF, z05, y_sk, u_sk05, x1, x2, x3 = core_vars
        sel_sites = []
        cap = dict(zip(sizes.k, sizes.cap_k))
        zip_s = dict(zip(sites.site_id, sites.zip_code_cleaned))
        for (s,k), var in y_sk.items():
            if var.X > 0.5:
                sel_sites.append((s, k, cap[k], zip_s.get(s, "")))
        print("\nSelected new sites:")
        if not sel_sites:
            print("  (none)")
        else:
            for s,k,capk,z in sel_sites:
                print(f"  site {s} ZIP {z} size {k} cap {capk}")
    else:
        print("[main] Model not solved to optimality. Status:", status, "SolCount:", m.SolCount)
        if status in [GRB.Status.INFEASIBLE, GRB.Status.INF_OR_UNBD]:
            print("[main] Model infeasible -> computing IIS...")
            m.computeIIS()
            infeas = [c.constrName for c in m.getConstrs() if c.IISConstr]
            print("[main] IIS constraints:", infeas[:20])

if __name__ == "__main__":
    main()


[load_data] ZIP intersection: 1375 common of (pop=1646, cls=1534)
[load_data] Dropped infeasible ZIPs: ['11219']
[load_data] areas: 1374, facilities (before filter): 15014, (after filter): 15013, sites: 215400
[conflict_pairs] conflict pairs found: 81383
[main] Building fairness model (Section 6)...
Set parameter OutputFlag to value 1
Set parameter TimeLimit to value 300
Set parameter MIPGap to value 0.005
[main] Optimizing fairness model...
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 25.0.0 25A362)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  300
MIPGap  0.005

Optimize a model with 1723833 rows, 1416628 columns and 7578827 nonzeros
Model fingerprint: 0x7821483a
Variable types: 770428 continuous, 646200 integer (646200 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+05]
  Objective range  [3e-01, 7e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e